In [1]:
from pulp import *
from itertools import product
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from tqdm import tqdm

#### Sets

In [2]:
%%time
#import files from github

V = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/V.csv') #vehicle types
F = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/F.csv') #fuel types
E = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/E.csv') #driving env
R = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/R.csv') #counties
M = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/M.csv') #charging stations
S = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/S.csv') #states

F['fuel_type'] = F['fuel_type'].apply(lambda x: x.replace('electricity','Electricity'))

CPU times: user 84.4 ms, sys: 11.1 ms, total: 95.5 ms
Wall time: 2.29 s


In [27]:
VEHICLE_TYPES = list(V['vehicle_type'])
FUEL_TYPES = list(F['fuel_type'])
DRIVING_ENV = list(E['driving_environment'])
COUNTIES = list(R['county'])
CHARGING_STATIONS = list(M['filling_stations'])
STATES = list(S['state'])

#### Parameters
Parameters describe objects statically, and is constant in a single simulation. Parameters are only changed to adjust model behaviour.

In [16]:
%%time
#EF(f,s): Emission factor for fuel type f in state s, in gallons/mile  
EF = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/EF(f%2Cs).csv')
EF['fuel_type'] = EF['fuel_type'].apply(lambda x: x.replace('electricity','Electricity'))
#FE(v,f):Average fuel economy for vehicle type v using fuel f
FE = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/FE(v%2Cf).csv')
FE['fuel_type'] = FE['fuel_type'].apply(lambda x: x.replace('electricity','Electricity'))
#C(f): Cost of fuel type f 
C = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/C(F).csv')
#CC (v,s): Capital cost of vehicle type v in state s
CC = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/CC(v%2Cs).csv') 
CC['yearly_cost'] = np.where(CC['vehicle_type']=='BEV', CC['cost_minus_rebate']/8.5,  CC['cost_minus_rebate']/5.5)
#BEVs are paid off in approx. 8-9 years, SIDI_ICE and FFV in avg 5.5 years
# CC['cost_minus_rebate'] = CC['cost_minus_rebate']/5.5
#CG: cost of fuel/gallon
CG = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/CG(F).csv')
CG['fuel_type'] = CG['fuel_type'].apply(lambda x: x.replace('electricity','Electricity'))
#D: Emission decrease goals per year
D = 0.25
#W(s):Current yearly GHG emissions per state
W = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/W(s).csv')
#TM (v, f, r): Total miles for vehicle v using fuel f in county r
TM = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/TM(f%2Cs).csv')
# # TM.drop(['household_income_ID'],axis=1,inplace=True)
#N(r): Average income per county  
N = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/N(r).csv')
# N.drop(['household_income_ID'],axis=1,inplace=True)
#B(r) county + state linking table
B = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/B(r).csv')
#CF(v,f): Fuel consumption for vehicle type v using fuel f (1/fuel economy)
CF = FE
CF['fuel_consumption'] = (1 / CF['fuel_economy'])


CPU times: user 123 ms, sys: 8.46 ms, total: 131 ms
Wall time: 1.23 s


,state,vehicle_type,capital_cost,state_rebate,federal_rebate,cost_minus_rebate,yearly_cost
0,CA,BEV,55000,2000,5000,48000,5647.058824
1,CA,SIDI_ICE,36000,0,0,36000,6545.454545
2,CA,FFV,36000,0,0,36000,6545.454545
3,MN,BEV,55000,250,5000,49750,5852.941176
4,MN,SIDI_ICE,36000,0,0,36000,6545.454545
5,MN,FFV,36000,0,0,36000,6545.454545
6,TX,BEV,55000,2500,5000,47500,5588.235294
7,TX,SIDI_ICE,36000,0,0,36000,6545.454545
8,TX,FFV,36000,0,0,36000,6545.454545


# Variables
Variables represent a model state and may change during simulation.

In [5]:
v = V['vehicle_type']
f = F['fuel_type']
r = R['county']

def variable_n():
    '''
    n(r,v,f) total optimal count of vehicle v using fuel type f in county r
    '''
    n = pd.DataFrame(list(product(r,v,f)), columns=['county', 'vehicle_type','fuel_type'])
    n['count'] = 0
    
    bev_elec = n[(n['vehicle_type']== 'BEV') & (n['fuel_type']== 'Electricity')]
    gas_e10 = n[(n['vehicle_type']== 'SIDI_ICE') & (n['fuel_type']== 'E10')]
    ffv_e85 = n[(n['vehicle_type']== 'FFV') & (n['fuel_type']== 'E85')]

    result = pd.concat([bev_elec,gas_e10,ffv_e85])
    result.sort_values(by=['county','vehicle_type'],inplace=True)
    result.reset_index(drop=True, inplace=True)
    return result

def variable_fc():
    '''
    fc(r,v,f) Total fuel consumption by vehicle v using fuel type f in county r
    '''
    fc = pd.DataFrame(list(product(r,v,f)), columns=['county','vehicle_type','fuel_type'])
    fc['fuel_consumption'] = 0
    
    bev_elec = fc[(fc['vehicle_type']== 'BEV') & (fc['fuel_type']== 'Electricity')]
    gas_e10 = fc[(fc['vehicle_type']== 'SIDI_ICE') & (fc['fuel_type']== 'E10')]
    ffv_e85 = fc[(fc['vehicle_type']== 'FFV') & (fc['fuel_type']== 'E85')]

    result = pd.concat([bev_elec,gas_e10,ffv_e85])
    result.sort_values(by=['county'],inplace=True)
    result.reset_index(drop=True, inplace=True)
    return result

def variable_oc():
    '''
    oc(r,v,f) Operating cost per mile for vehicle v using fuel type f in county r 
    '''
    oc = pd.DataFrame(list(product(r,v,f)), columns=['county', 'vehicle_type','fuel_type'])
    oc['operating_cost'] = 0
    
    bev_elec = oc[(oc['vehicle_type']== 'BEV') & (oc['fuel_type']== 'Electricity')]
    gas_e10 = oc[(oc['vehicle_type']== 'SIDI_ICE') & (oc['fuel_type']== 'E10')]
    ffv_e85 = oc[(oc['vehicle_type']== 'FFV') & (oc['fuel_type']== 'E85')]

    result = pd.concat([bev_elec,gas_e10,ffv_e85])
    result.sort_values(by=['county'],inplace=True)
    result.reset_index(drop=True, inplace=True)
    return result 

def variable_tac():
    '''
    tac(r,v,f) Total annual cost of vehicle v using fuel type f in county r 
    '''
    tac = pd.DataFrame(list(product(r,v,f)), columns=['county', 'vehicle_type','fuel_type'])
    tac['total_annual_vehicle_cost'] = 0
    
    bev_elec = tac[(tac['vehicle_type']== 'BEV') & (tac['fuel_type']== 'Electricity')]
    gas_e10 = tac[(tac['vehicle_type']== 'SIDI_ICE') & (tac['fuel_type']== 'E10')]
    ffv_e85 = tac[(tac['vehicle_type']== 'FFV') & (tac['fuel_type']== 'E85')]

    result = pd.concat([bev_elec,gas_e10,ffv_e85])
    result.sort_values(by=['county'],inplace=True)
    result.reset_index(drop=True, inplace=True)
    return result

def variable_ce():
    '''
    ce(r,v,f)  GHG emission per year of vehicle v using fuel type f in county r 
    '''
    ce = pd.DataFrame(list(product(r,v,f)), columns=['county', 'vehicle_type','fuel_type'])
    ce['emission_per_year'] = 0
    
    bev_elec = ce[(ce['vehicle_type']== 'BEV') & (ce['fuel_type']== 'Electricity')]
    gas_e10 = ce[(ce['vehicle_type']== 'SIDI_ICE') & (ce['fuel_type']== 'E10')]
    ffv_e85 = ce[(ce['vehicle_type']== 'FFV') & (ce['fuel_type']== 'E85')]

    result = pd.concat([bev_elec,gas_e10,ffv_e85])
    result.sort_values(by=['county'],inplace=True)
    result.reset_index(drop=True, inplace=True)
    return result

n = variable_n()
fc = variable_fc()
tac = variable_tac()
oc = variable_oc()
ce = variable_ce()

## IEMS 394 - Biofuels Optimisation Model

Running list of assumptions:
- Some county name recurr within the set of states we have selected. Such counties have had their respective 2-letter state code appended to their name. They are enumerated below.
    1. Orange - CA,TX 
    2. Cass - MN, TX
    3. Lake - CA, MN
    4. Trinty - CA, TX
    5. Houston - MN, TX
    6. Polk - MN,TX
    7. Brown - MN,TX
    8. Clay - MN, TX
    9. Jackson - TX,MN
    10. Washington - MN, TX
    11. Martin - MN,TX
    
- future proofing changes:
    - Decision variable `n` assumes that vehicle types use a single type of fuel (it's not written in at all)

In [6]:
n1 = n.merge(B)
n2 = n1.merge(CG)
n3 = n2.merge(oc)
n4 = n3.merge(ce)
n5 = n4.merge(fc)
n6 = n5.merge(tac)
n6 = n6.sort_values(by=['county','vehicle_type'], ascending=True)
n6.reset_index(drop=True, inplace=True)

In [7]:
N_B = B.merge(N)
number_one = N_B.merge(TM, left_on=['household_income_ID','state'],right_on=['household_income_ID','state'])
number_one.drop(['household_income','household_income_ID'],axis=1,inplace=True)
n6 = n6.merge(number_one)
total_miles_df = n6[['county','vehicle_type','annual_miles_driven']]

# Parameters

In [10]:
EF_param = {k: f.groupby('fuel_type')['emission_factor'].apply(list).to_dict()
     for k, f in EF.groupby('state')}

FE_param = {k: f.groupby('fuel_type')['fuel_economy'].apply(list).to_dict()
     for k, f in FE.groupby('vehicle_type')}

C_param = {k: f.groupby('fuel_type')['fuel_cost_per_mile'].apply(list).to_dict()
     for k, f in C.groupby('state')}
# CC_param = {k: f.groupby('vehicle_type')['cost_minus_rebate'].apply(list).to_dict()
#      for k, f in CC.groupby('state')}
improve = CC.merge(B)
CC_param = {k: f.groupby('vehicle_type')['cost_minus_rebate'].apply(list).to_dict()
     for k, f in improve.groupby('county')}
CC_param

CG_param = {k: f.groupby('fuel_type')['fuel_cost_per_gal'].apply(list).to_dict()
     for k, f in CG.groupby('state')}
D_param = 0.25

W_param = {'CA': [3.610000e+14],
 'MN': [8.930000e+13],
 'TX': [6.540000e+14]}
TM_param = {k: f.groupby('vehicle_type')['annual_miles_driven'].apply(list).to_dict()
     for k, f in total_miles_df.groupby('county')}
# CF_param = {k: f.groupby('fuel_type')['fuel_economy'].apply(list).to_dict()
#      for k, f in CF.groupby('vehicle_type')}
CFnew = CF.merge(n6,left_on="vehicle_type",right_on="vehicle_type")
CFnew.sort_values(by=['county'],inplace=True)
CFnew.reset_index(drop=True, inplace=True)
CFnew = CFnew[['county','vehicle_type','fuel_consumption_x']]
CFnew_param = {k: f.groupby('vehicle_type')['fuel_consumption_x'].apply(list).to_dict()
     for k, f in CFnew.groupby('county')}

CGnew_param = n6[['county','vehicle_type','fuel_cost_per_gal']]
CGnew_param = {k: f.groupby('vehicle_type')['fuel_cost_per_gal'].apply(list).to_dict()
     for k, f in CGnew_param.groupby('county')}

T = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/T(r).csv')
T['total_vehicles_registered'] = T['total_vehicles_registered'].apply(lambda x: x.replace(',',''))
T['total_vehicles_registered'] = T['total_vehicles_registered'].apply(pd.to_numeric)
T = T.set_index('county').T.to_dict('int')
# access using T['total_vehicles_registered'][r]...



KeyError: ('cost_minus_rebate', 'BEV')

In [9]:
#decision variables
#n (v, f, r) 
#Projected number of vehicles of vehicle type v using fuel type f that should be in county r
n = LpVariable.dicts("vehicle_count", (COUNTIES, VEHICLE_TYPES), )

#fc(r,v,f) 
#Total fuel consumption by vehicle v using fuel type f in county r
# fc = LpVariable.dicts('total_fuel_use',[(r,f) for r in COUNTIES for f in VEHICLE_TYPES], cat = 'Integer')
fc = LpVariable.dicts("total_fuel_use", (COUNTIES, VEHICLE_TYPES), 0)

#oc(r,v,f) 
#Operating cost per mile for vehicle v using fuel type f in county r 
# oc = LpVariable.dicts('operating_cost_permile',[(r,f) for r in COUNTIES for f in VEHICLE_TYPES], cat = 'Integer')
oc = LpVariable.dicts("operating_cost_permile", (COUNTIES, VEHICLE_TYPES), 0)

#tac(r,v,f) 
#Total annual cost of vehicle v using fuel type f in county r 
# tac = LpVariable.dicts('total_annual_cost',[(r,f) for r in COUNTIES for f in VEHICLE_TYPES], cat = 'Integer')
tac = LpVariable.dicts("total_annual_cost", (COUNTIES, VEHICLE_TYPES), 0)

#ce(r,v,f)  
#GHG emission per year of vehicle v using fuel type f in county r
# ce = LpVariable.dicts('GHG_emissions_new',[(r,f) for r in COUNTIES for f in VEHICLE_TYPES], cat = "Integer")
ce = LpVariable.dicts("GHG_emissions_new", (COUNTIES, VEHICLE_TYPES), 0)

# Objective Function

In [10]:
# def objective():
#     '''
#     Minimize the total cost per mile driven in all of US (which includes the capital cost and operating cost)
    
#     capital fuel cost + ((cost of fuel per gallon of vehicle)(fuel consumption of vehicle/per mile))*(total miles traveled)
#     Sum this over counties and then over states
#     '''    
#     result = []
#     for i in tqdm(range(n6.shape[0])):
#         result.append((n6['count'][i] * (n6['fuel_consumption'][i] * n6['fuel_cost_per_gal'][i] * n6['annual_miles_driven'][i])))
#     return list(np.add.reduceat(result, np.arange(0, len(result), 3)))
# # for each county
# # for each vehicle type
# # compute capital cost + ((cost of fuel/gallon) * fuel consumption 1/mpg * miles_driven)
# # so this comes out to CC + (CG * CF * miles_driven)
# # cost vector = vehicle type * (CC +(CG * CF * miles_driven))



#OBJECTIVE FUNCTION: minimise cost, determine car allocation (mixed integer)
# prob += lpSum(CC_param[r][f] * n[(r,f)] for r in COUNTIES for f in VEHICLE_TYPES)
#SET PROBLEM VARIABLE
prob = LpProblem('Biofuels',LpMinimize)

# for r in COUNTIES:
#     prob += lpSum(((CFnew_param[r][f][0] * CGnew_param[r][f][0] * TM_param[r][f][0]) + CC_param[r][f][0]) * n[r][f] for f in VEHICLE_TYPES)
prob += lpSum(((CFnew_param[r][f][0] * CGnew_param[r][f][0] * TM_param[r][f][0]) + CC_param[r][f][0]) * n[r][f] for f in VEHICLE_TYPES for r in COUNTIES)

In [11]:
len(COUNTIES)

399

# Constraints

In [12]:
v = V['vehicle_type']
f = F['fuel_type']
r = R['county']
CE = variable_ce()
ce_update = CE.merge(B)
ce = {k: f.groupby('county')['emission_per_year'].apply(list).to_dict()
     for k, f in ce_update.groupby('state')}

In [13]:
#Constraint 0: Non-negative vehicles assigned to each county r 
for r in COUNTIES:
    prob += lpSum(n[r][f] for f in VEHICLE_TYPES) >= 0

In [14]:
#Constraint 1: Annual emission by total of vehicle v in county f with a given fuel ce(v,f) equals emission per mile * total miles driven  
#This needs to run before Constraint 2 in the final compilation
Nnew = N.merge(B)
EFnew = pd.merge(TM, Nnew,  how='left', left_on=['state','household_income_ID'], right_on = ['state','household_income_ID']).dropna()
EFnew = EFnew.merge(EF).merge(FE)
EFnew.sort_values(by=['county'],inplace=True)
EFnew.reset_index(drop=True, inplace=True)
EFnew['TotalEmission'] = EFnew['annual_miles_driven']*EFnew['emission_factor']
EFnew
#action items: change it to columns of three
#change fuel type to vehicle type
#change units to grams/mile

#EF_param = {k: f.groupby('vehicle_type')['emission_factor'].apply(list).to_dict()
#     for k, f in EF.groupby('state')}
#EF_param
EFnew_param = {k: f.groupby('vehicle_type')['TotalEmission'].apply(list).to_dict()
     for k, f in EFnew.groupby('county')}
# EFnew_param

# for r in COUNTIES:
#     for f in VEHICLE_TYPES:
#         prob +=  ce[r][f] == EFnew_param[r][f][0] * n[r][f]

In [15]:
#Constraint 2: Decrease total emissions by D(s) for each state

value_states = []
for i in STATES:
    value_states.append(list(ce[i].values())) #value_states is a list of lists of states and their counties 
    
states_ghg = []
for i in range(len(value_states)): #for each state
    state_sum = [] #set it so zero to start a new count for each state
    for j in range(len(value_states[i])): #for each county
        state_sum.append(sum(value_states[i][j]))
    states_ghg.append(sum(state_sum))

statesghg_dict = dict(zip(('CA', 'MN','TX'), states_ghg))
statesghg_dict['CA']

for i in STATES:
    prob += lpSum(statesghg_dict[i]) <= W_param[i][0]*(1-D)

In [16]:
#Constraints 3: E85 Viability index
#Constraint 4: EV Viability index

In [17]:
CFnew_param['Alameda']['BEV'][0]

0.009167863554173324

In [18]:
#Constraint 5: Total annual fuel consumption per county equals fuel consumption of each vehicle in that county
for r in COUNTIES:
    for f in VEHICLE_TYPES:
        prob += fc[r][f]  - (CFnew_param[r][f][0] * n[r][f] * TM_param[r][f][0]) == 0
    

In [19]:
#Constraint 6: Operating cost= cost of fuel * sum of fuel consumption
for r in COUNTIES:
    for f in VEHICLE_TYPES:
        prob += oc[r][f] - (CGnew_param[r][f][0] * fc[r][f]) == 0

In [20]:
# Constraint 7: Total of number of all vehicle types v should be equal to the total of all vehicles in the county


In [22]:
prob.solve() #1 if it works

1

In [31]:
# print("The Min Value = ",value(prob.objective))
# print(prob)
# # print(n)
# cost vector = vehicle type * (CC +(CG * CF * miles_driven))

- Flesh out the objective function with additional parameters
    - why only 1 county?
    - cost vector = vehicle type * (CC +(CG * CF * miles_driven)) # saif
- Develop constraints
    - infrastructure constraints?
    - 

In [ ]:
for i in STATES:
    prob += lpSum(variable_ce[r][i] for r in COUNTIES) <= (1-D)*W[i]

In [ ]:
# convert b into dictionary where it's state:county as key:value
#for i in STATES: ... where STATES = ['CA','MN','TX']
#B_dict[i] # all of CA's counties ... MN... TX
B_dict = B.to_dict('records')

# for i in range(len(COUNTIES)):
#     print(B_dict[i]['county'])
#     print(B_dict[i]['state'])